In [2]:
import os
import json
import pickle
from tqdm import tqdm
import pandas
from embeddings import get_embeddings
import multiprocessing as mp
from datasets import load_dataset

autocast_df = pandas.DataFrame(json.load(open('../../autocast_questions.json')))
test_questions = json.load(open('../../autocast_competition_test_set.json'))
test_df = pandas.DataFrame(test_questions)
test_ids = [q['id'] for q in test_questions]
cc_news = load_dataset('cc_news')
autocast_df.head()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Found cached dataset cc_news (/Users/eamon/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/ae469e556251e6e7e20a789f93803c7de19d0c4311b6854ab072fecb4e4

,question,id,background,publish_time,close_time,tags,source_links,prediction_count,forecaster_count,answer,choices,status,qtype,crowd
0,What will the end-of-day closing value for the...,G1,Outcome will be determined by the end-of-day c...,2015-09-01 13:49:29.860000+00:00,2016-01-01 17:00:01+00:00,"[Finance, Economic Indicators]",[http://ftalphaville.ft.com/2015/08/17/2137329...,1549,385,D,"[Less than 6.30, Between 6.30 and 6.35, inclus...",Resolved,mc,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
1,How many seats will the Justice and Developmen...,G2,The Justice and Development Party (AKP) failed...,2015-09-01 13:54:25.050000+00:00,2015-11-01 22:00:20+00:00,"[Elections and Referenda, Non-US Politics]",[http://www.al-monitor.com/pulse/originals/201...,567,194,A,"[A majority, A plurality, Not a plurality]",Resolved,mc,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
2,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30.138000+00:00,2015-11-30 14:00:15+00:00,[Finance],[http://atimes.com/2015/11/china-will-allow-su...,545,148,yes,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
3,Will the Export-Import Bank of the United Stat...,G5,The Export-Import Bank's authorization expired...,2015-09-01 14:02:21.242000+00:00,2015-12-04 14:00:25+00:00,"[Economic Policy, US Politics, US Policy]",[http://thehill.com/policy/finance/260118-week...,1000,379,yes,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
4,Will a trilateral meeting take place between C...,G6,"A trilateral meeting of leaders from China, Ja...",2015-09-01 14:04:41.470000+00:00,2015-12-31 23:00:11+00:00,[Foreign Policy],"[https://en.wikipedia.org/wiki/Li_Keqiang, htt...",946,385,no,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."


# Embeddings

In [3]:
# Tokenize the question (use multiprocessing and tqdm)
embeddings = {}
with mp.Pool(mp.cpu_count()) as pool:
    for i, embedding in tqdm(enumerate(pool.imap(get_embeddings, autocast_df['question'].values)), total=len(autocast_df)):
        embeddings[i] = embedding

# Save to df
autocast_df['embeddings'] = embeddings

# Save the disk
with open('ac_embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

  0%|          | 0/6532 [00:00<?, ?it/s]Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_rela

## Load Saved Embeddings

In [ ]:
with open('autocast_embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)
    autocast_df['embedding'] = embeddings

## Query Embeddings for cc_news

In [ ]:
from tokenizer import get_embeddings
import multiprocessing as mp
# Tokenize the question (use multiprocessing and tqdm)
embeddings = {}
with mp.Pool(mp.cpu_count()) as pool:
    for i, embedding in tqdm(enumerate(pool.imap(get_embeddings, cc_news['train']['text'])), total=len(cc_news['train']['text'])):
        embeddings[i] = embedding

# Save to dataset
cc_news['train']['embedding'] = embeddings

# Save the dataset to disk
cc_news.save_to_disk('cc_news_embeddings')

# Save the disk
with open('cc_embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.trans